# Infinite Mixture Models with Non-Parametric Bayes and the Dirichlet Process

Imagine you're a budding chef. You start taking a set of foods and ask 10 friends how much of each they ate yesterday. Your goal is to find natural groups of foodies, so that you can better cater to each cluster's tastes. Some will like wings and beer, others will like soba and sushi, some will like tofu only, and so on.

How can you use the data you've gathered to discover different kinds of groups?

![Three possible clusterings of a synthetic data set.](http://i.imgur.com/sxBfR1R.png)

One way is to use a standard clustering algorithm like k-means or Gaussian mixture modeling, but the problem with these is that both assume a fixed number of clusters, which they need to be told to find.

There are a couple methods for selecting the number of clusters to learn (e.g. gap and prediction strength statistics), but the problem is a more fundamental one: most real-world data simply does not have a fixed number of clusters.

Suppose we've asked 10 of our friends what they are in the past day, and we want to find groups of eating preferences. There's really an infinite number of foodie types (carnivore, vegan, snacker, Italian-food-only, fast-food-only, and so on), but with only 10 friends, we simply don't ave enough data to detect them all. Indeed, we'd be limited to only 10 clusters!

So whereas k-means starts with the incorrect assumption that there's a fixed, finite number of clusters that our points come from, no matter if we feed it more data, what we'd really like is a mehod positing an infinite number of hidden clusters that naturally arise as we ask more friends about their food habits.

For example, with only 2 data points, we might not be able to tell the difference between vegans and vegetarians, but with 200 data points, we probably could.

Luckily for us, this is precisely the purview of nonparametric Bayes, which allows some parameters to change with the data.

## A Generative Story

Let's describe a generative model for finding clusters in any set of data. We assume an infinite set of latent groups, where each group is described by some set of parameters. For example, each group could be a Gaussian with a specified mean $\mu_{i}$ and standard deviation $\sigma_{i}$, and these group parameters themselves are assumed to come from some base distribution $G_{0}$. Data is then generated in the following manner:

* Select a cluster.
* Sample from that cluster to generate a new point.

(Note the resemblance to the finite fixture model.)

For example, suppose we ask 10 friends how many calories of pizza, salad, and rice they ate yesterday. Our groups could be:

* A Gaussian centered at (pizza = 5000, salad = 100, rice = 500)
* A Gaussian centered at (pizza = 100, salad = 3000, rice = 1000)
* A Gaussian centered at (pizza = 100, salad = 100, rice = 10000)
* ...

When deciding what to eat when she woke up yesterday, Alice could have easily been in the mood for pizza and her food consumption yesterday would have been a sample from the pizza Gaussian. Similarly, Bob could have sampled from the rice Gaussian for his day's meals, and so on.

The big question, then, is: how do we assign each friend to a group?

## Assigning Groups

### Chinese Restaurant Process

Imagine a restaurant where all your friends went to eat yesterday:

* Initially the restaurant is empty.
* The first person to enter sits down at a table. They then order good for the table, and everyone else who joins the table will then be limited to eating from the food that they ordered.
* The second person to enter sits down at a table. Which table do they sit at? With probability $\dfrac{\alpha}{(1 + \alpha)}$, they sit down at a new table and orders food for the table, with probability $\dfrac{1}{(1 + \alpha)}$ that they sit down with the first person and eats from the food they've already ordered.
* ...
* The $(n + 1$th person sits down at a new table with probability $\dfrac{\alpha}{(n + \alpha)}$ and at table $k$ with probability $\dfrac{n_{k}}{n + \alpha}$, where $n_{k}$ is the number of people currently sitting at table $k$.

Note:

* The more people there are at a table, the more likely it is that people will join it.
* There is always a small probability that someone sits at an entirely new table.
* The probability of a new table depends on $\alpha$, which is inversely proportional to the clustering of our guests.

Also note the resemblance between table selection probabilities and a Dirichlet distribution.

In [3]:
from random import random, seed

In [7]:
def ChineseRestaurantProcess(num_of_customers, alpha):
    if num_of_customers == 0:
        return []
    
    table_assignments = [1] # First customer sits at table 1.
    next_open_table = 2 # Index of next empty table.
    
    for i in range(1, num_of_customers - 1):
        if random() < (alpha / (alpha / i)):
            # Customer sits at a new table.
            table_assignments = next_open_table
            next_open_table += 1
        else:
            # Customer sits at an existing table.
            which_table = table_assignments[random() * table_assignments.size]
            table_assignments = which_table
            
    return table_assignments

In [8]:
ChineseRestaurantProcess(num_of_customers = 10, alpha = 1)

9